In [1]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import numpy as np
import random
from pyngrok import ngrok

# Dash 앱 초기화
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server

# -----------------------------
# (1) 데이터 생성
# -----------------------------
np.random.seed(42)
spots = [
    '팝업스토어 A', '팝업스토어 B', '서울숲 공연장', '연무장길 카페거리', '성수역 부근',
    '성수동 복합문화공간', '성수 창작소', '거리버스킹 ZONE', '성수동 야시장', '한강진입로 쉼터'
]
timeslots = ['10:00', '13:00', '15:00', '17:00', '19:00']
data = []
for time in timeslots:
    for spot in spots:
        lat = 37.544 + np.random.uniform(-0.003, 0.003)
        lon = 127.056 + np.random.uniform(-0.003, 0.003)
        popscore = random.randint(30, 100)
        crowd = random.randint(50, 700)
        card_spend = random.randint(100000, 2000000)
        traffic_flow = random.randint(30, 300)
        tag = random.choice(['#팝업', '#공연', '#혼잡', '#무질서', '#축제', '#정상'])
        data.append({
            'spot': spot, 'time': time, 'lat': lat, 'lon': lon,
            'popscore': popscore, 'crowd': crowd, 'card_spend': card_spend,
            'traffic': traffic_flow, 'tag': tag
        })
df = pd.DataFrame(data)
time_now = '15:00'
logs = []

# -----------------------------
# (2) 페이지 템플릿 함수들
# -----------------------------
def home_page():
    return html.Div([
        html.H2("성수 SPOT 혼잡 예측 대시보드", className="text-center text-primary mt-4"),
        html.P("사용자 유형을 선택해주세요.", className="text-center text-muted mb-4"),
        html.Ul([
            html.Li(html.A("📦 팝업운영자용 →", href="/popup")),
            html.Li(html.A("🧭 관광안내사용 →", href="/guide")),
            html.Li(html.A("🛠️ 총괄관리자용 →", href="/admin"))
        ], className="list-unstyled text-center")
    ], className="container")

def make_dashboard(user_role):
    return html.Div([
        html.H3(f"🔍 {user_role} 대시보드", className="text-center mt-4"),
        html.P("성수동의 혼잡도 예측 시각화 및 SPOT 선택", className="text-center text-muted"),
        html.Div([
            html.Label("시간대 선택"),
            dcc.Dropdown(
                id='time-selector',
                options=[{'label': t, 'value': t} for t in timeslots],
                value=time_now,
                clearable=False
            ),
            html.Label("SPOT 선택"),
            dcc.Dropdown(id='spot-selector')
        ], className="container mb-3"),
        dcc.Graph(id='spot-map'),
        html.Div([
            html.H5("⏱️ 로그"),
            html.Div(id='log-output', className='bg-light p-3 rounded shadow-sm')
        ], className="container")
    ])

# -----------------------------
# (3) App Layout 설정
# -----------------------------
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content', children=home_page())
])

# -----------------------------
# (4) 라우팅 콜백
# -----------------------------
@app.callback(Output('page-content', 'children'), Input('url', 'pathname'))
def route_page(pathname):
    if pathname in ['/', None]:
        return home_page()
    elif pathname == '/popup':
        return make_dashboard("팝업운영자")
    elif pathname == '/guide':
        return make_dashboard("관광안내사")
    elif pathname == '/admin':
        return make_dashboard("총괄 관리자")
    else:
        return html.Div([
            html.H3("❌ 잘못된 경로입니다."),
            html.A("홈으로 돌아가기", href="/")
        ], className="container")

# -----------------------------
# (5) 시간대 선택 → 지도 + SPOT 리스트 업데이트
# -----------------------------
@app.callback(
    [Output('spot-map', 'figure'), Output('spot-selector', 'options'), Output('spot-selector', 'value')],
    Input('time-selector', 'value')
)
def update_map_and_spotlist(time_selected):
    import plotly.express as px
    df_filtered = df[df['time'] == time_selected]
    fig = px.scatter_mapbox(
        df_filtered, lat='lat', lon='lon', hover_name='spot',
        hover_data={'crowd': True, 'popscore': True, 'card_spend': True, 'traffic': True, 'tag': True},
        color='crowd', size='crowd', zoom=14, height=600,
        color_continuous_scale='YlOrRd'
    )
    fig.update_layout(mapbox_style="carto-positron", margin={"r":0,"t":0,"l":0,"b":0})
    options = [{'label': s, 'value': s} for s in df_filtered['spot'].unique()]
    return fig, options, options[0]['value']

# -----------------------------
# (6) SPOT 선택 → 강조 표시
# -----------------------------
@app.callback(
    Output('spot-map', 'figure'),
    [Input('spot-selector', 'value'), Input('time-selector', 'value')]
)
def highlight_selected_spot(selected_spot, time_selected):
    import plotly.express as px
    df_filtered = df[df['time'] == time_selected]
    fig = px.scatter_mapbox(
        df_filtered, lat='lat', lon='lon', hover_name='spot',
        hover_data={'crowd': True, 'popscore': True, 'card_spend': True, 'traffic': True, 'tag': True},
        color='crowd', size='crowd', zoom=14, height=600,
        color_continuous_scale='YlOrRd'
    )
    selected_row = df_filtered[df_filtered['spot'] == selected_spot]
    if not selected_row.empty:
        fig.add_trace(px.scatter_mapbox(
            selected_row, lat='lat', lon='lon', size=[150],
            marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=6))
        ).data[0])
        fig.update_layout(
            mapbox_center={'lat': selected_row.iloc[0]['lat'], 'lon': selected_row.iloc[0]['lon']},
            mapbox_zoom=15
        )
    fig.update_layout(mapbox_style="carto-positron", margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# -----------------------------
# (7) 로그 출력
# -----------------------------
@app.callback(
    Output('log-output', 'children'),
    Input('spot-selector', 'value')
)
def update_log(selected_spot):
    from datetime import datetime
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    logs.append(f"[{now}] {selected_spot} 선택됨.")
    return [html.P(l) for l in logs[::-1]]

# -----------------------------
# (8) ngrok 실행
# -----------------------------
port = 8050
public_url = ngrok.connect(port)
print(f"🌐 외부 접속 주소: {public_url}")

# -----------------------------
# (9) 앱 실행
# -----------------------------
app.run_server(port=port)


ImportError: cannot import name 'url_quote' from 'werkzeug.urls' (/usr/local/lib/python3.12/dist-packages/werkzeug/urls.py)

In [10]:
!ngrok config add-authtoken 30obiR8dDAqUXi34iipzdcjnHSC_2NtQ6kUYsFTDqKid4HQWK

Authtoken saved to configuration file: /home/work/.config/ngrok/ngrok.yml


In [51]:
streamlit run streamlit_app.py


SyntaxError: invalid syntax (2482160671.py, line 1)

In [49]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg a